In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [21]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score , mean_squared_error , mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor , AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression , Ridge , Lasso
from sklearn.model_selection import RandomizedSearchCV , train_test_split
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import warnings

In [3]:
df = pd.read_csv("./data/StudentsPerformance.csv")
df.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [4]:
X = df.drop("math score" , axis=1)

In [5]:
y = df["math score"]

In [8]:
num_features = X[["reading score" , "writing score"]].columns
num_features

Index(['reading score', 'writing score'], dtype='object')

In [9]:
df.columns

Index(['gender', 'race/ethnicity', 'parental level of education', 'lunch',
       'test preparation course', 'math score', 'reading score',
       'writing score'],
      dtype='object')

In [10]:
cat_features = X[["gender" , "lunch" , "parental level of education" , "race/ethnicity" , "test preparation course"]].columns
cat_features


Index(['gender', 'lunch', 'parental level of education', 'race/ethnicity',
       'test preparation course'],
      dtype='object')

In [11]:
from sklearn.preprocessing import OneHotEncoder , StandardScaler
ss = StandardScaler()
ohe = OneHotEncoder()
from sklearn.compose import ColumnTransformer

preprocessor = ColumnTransformer([
    ("one hot encoder" , ohe , cat_features),
    ("standardisation" , ss , num_features)
])

In [12]:
X_train , X_test , y_train , y_test = train_test_split(X , y , test_size=0.25 , random_state=42)

In [13]:
X_train = preprocessor.fit_transform(X_train)

In [27]:
X_test = preprocessor.transform(X_test)

In [30]:
def evaluate_model(true , predicted):
    mse = mean_squared_error(true , predicted)
    mae = mean_absolute_error(true , predicted)
    score = r2_score(true,predicted)
    rmse = np.sqrt(mse)
    return rmse , mae , score


In [22]:
models = {
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
    "XGBRegressor": XGBRegressor(), 
    "CatBoosting Regressor": CatBoostRegressor(verbose=False),
    "AdaBoost Regressor": AdaBoostRegressor()
}

In [31]:
model_list = []
r2_list =[]

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train) # Train model

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Evaluate Train and Test dataset
    model_train_rmse , model_train_mae, model_train_r2 = evaluate_model(y_train, y_train_pred)

    model_test_mae , model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    r2_list.append(model_test_r2)
    
    print('='*35)
    print('\n')

Linear Regression
Model performance for Training set
- Root Mean Squared Error: 5.2972
- Mean Absolute Error: 4.2383
- R2 Score: 0.8743
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 4.7442
- Mean Absolute Error: 6.0423
- R2 Score: 0.8516


Lasso
Model performance for Training set
- Root Mean Squared Error: 6.5469
- Mean Absolute Error: 5.1796
- R2 Score: 0.8080
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 5.7471
- Mean Absolute Error: 7.2360
- R2 Score: 0.7872


Ridge
Model performance for Training set
- Root Mean Squared Error: 5.2975
- Mean Absolute Error: 4.2369
- R2 Score: 0.8743
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 4.7423
- Mean Absolute Error: 6.0409
- R2 Score: 0.8517


K-Neighbors Regressor
Model performance for Training set
- Root Mean Squared Error: 5.7483
- Mean Absolute Error: 4.5477
- R2 Score: 0.8520
-----------------------

In [32]:
pd.DataFrame(list(zip(model_list, r2_list)), columns=['Model Name', 'R2_Score']).sort_values(by=["R2_Score"],ascending=False)

,Model Name,R2_Score
2,Ridge,0.851670
0,Linear Regression,0.851604
7,CatBoosting Regressor,0.828517
5,Random Forest Regressor,0.812611
8,AdaBoost Regressor,0.810901
6,XGBRegressor,0.797673
1,Lasso,0.787174
3,K-Neighbors Regressor,0.759712
4,Decision Tree,0.724677
